In [11]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path, PureWindowsPath

In [12]:
# constants  #
#------------#
HDF5_dir = "C:\MessdatenStoeckach\\"
resampled_dir = r'C:\resampled'
dates = os.listdir(HDF5_dir)
phases = ["0","1","2","3"]

In [3]:
def groupedAvg(myArray, N=2):
    result = np.cumsum(myArray, 0)[N-1::N]/float(N)
    result[1:] = result[1:] - result[:-1]
    return result

In [10]:
%%time
for date in dates:
    print(date)
    with h5py.File(HDF5_dir + date + "\\" + "harmonics_phase_" + "0" + "_" + date + ".h5", 'r') as f:
            timestamps = np.array(f["tenperiodtable"]["timestamp"])
   
    newLength = len(timestamps)//3000
    dayly_aggregated_u = np.zeros([newLength,2960])
    dayly_aggregated_i = np.zeros([newLength,2960])
    dayly_max_u = np.zeros([0,2960])
    dayly_max_i = np.zeros([0,2960])
    
    for phase in phases:
        print(phase + "_u")
        with h5py.File(HDF5_dir + date + "\\" + "harmonics_phase_" + phase + "_" + date + ".h5", 'r') as f:
            sh_u = np.array(f["tenperiodtable"]["harmonics_u"][:,38:], dtype=np.float32)
        sh_agg_u = groupedAvg(sh_u, 3000)
        sh_u_max = np.amax(sh_u, axis=0)
        dayly_aggregated_u += sh_agg_u
        dayly_max_u = np.vstack([dayly_max_u, sh_u_max])
        del sh_u
        del sh_agg_u
        del sh_u_max
    dayly_agg_u_df = pd.DataFrame((dayly_aggregated_u/4), index=timestamps[3000::3000])
    dayly_agg_u_df.to_excel(resampled_dir + "\\" + "u" + "\\" + date + "_harmonics10min_u.xlsx")  
    pd.DataFrame(np.amax(dayly_max_u, axis=0)).to_excel(resampled_dir + "\\" + "max" + "\\" + "u" + "\\" + date + "_max_u.xlsx")
    
    for phase in phases:
        print(phase + "_i")
        with h5py.File(HDF5_dir + date + "\\" + "harmonics_phase_" + phase + "_" + date + ".h5", 'r') as f:
            sh_i = np.array(f["tenperiodtable"]["harmonics_i"][:,38:], dtype=np.float32)
        sh_agg_i = groupedAvg(sh_i, 3000)
        sh_i_max = np.amax(sh_i, axis=0)
        dayly_aggregated_i += sh_agg_i
        dayly_max_i = np.vstack([dayly_max_i, sh_i_max])
        del sh_i
        del sh_agg_i
        del sh_i_max
    dayly_agg_i_df = pd.DataFrame((dayly_aggregated_i/4), index=timestamps[3000::3000])
    dayly_agg_i_df.to_excel(resampled_dir + "\\" + "i" + "\\" + date + "_harmonics10min_i.xlsx")  
    pd.DataFrame(np.amax(dayly_max_u, axis=0)).to_excel(resampled_dir + "\\" + "max" + "\\" + "i" + "\\" + date + "_max_i.xlsx")
    

2017-04-10
0_u
1_u
2_u
3_u
0_i
1_i
2_i
3_i
2017-04-21
0_u
1_u
2_u
3_u
0_i
1_i
2_i
3_i
Wall time: 32min 8s
